In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')

In [2]:
#importing all the required packages
import optuna

import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
import os
from pathlib import Path

from model.GRU_model import *
from pipeline_config import *
from utils import *

/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.075
Epochs : 100


In [4]:
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [5]:
weightspath

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/models_weights/'

In [6]:
def objective(trial):
    with open(train_dataset_path, 'rb') as f:
        train_dataset_dict = pickle.load(f)
        
    with open(test_dataset_path, 'rb') as f:
        test_dataset_dict = pickle.load(f)

    with open(category_id_to_category_name_path, 'rb') as f:
        category_id_to_name_dict = pickle.load(f)
        
    with open(categories_per_indent_path, 'rb') as f:
        categories_per_indent_dict = pickle.load(f)

    with open(son_parent_path, 'rb') as f:
        son_parent_dict = pickle.load(f)

    with open(parent_to_son_list_path, 'rb') as f:
        parent_to_son_list_dict = pickle.load(f)

    with open(hgru_model_weights_path, 'rb') as f:
        hgru_weight_dict = pickle.load(f)

    with open(coefficient_dict_path, 'rb') as f:
        coefficient_dict = pickle.load(f)
    
    weights_path = weightspath

    #--------------------------------------------------------------------------------------------------------------------------------------#

    loss_coef_1 = trial.suggest_float('loss_coef_1',  1e-15, 1e-6, log=True)
    loss_coef_2 = trial.suggest_float('loss_coef_2',  1e-15, 1e-6, log=True)
    Lr = trial.suggest_float('Lr',  1e-10, 1e-1, log=True)

    bi_directional_models = {}
    num_categories = 0

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')
            #if category_name == 'Food at home': #for now...
            #    continue

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
 
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            min_error = training_and_evaluation_optuna(trial, model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, alpha, saving_param_path)
            
            bi_directional_models[category] = min_error

    average_error = sum(list(bi_directional_models.values()))/len(list(bi_directional_models.values()))
    
    return average_error


In [7]:
EPOCHS = 50
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

[I 2024-09-07 15:35:45,200] A new study created in memory with name: no-name-9f31007d-58d0-49c7-812f-ab094771aa40


num categories: 1
category id|name: 9540.0|Bacon and related products
------------------------------------------------------------------
num categories: 2
category id|name: 8302.0|Breakfast sausage and related products
------------------------------------------------------------------


/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python

num categories: 3
category id|name: 2135.0|Fresh and frozen chicken parts
------------------------------------------------------------------
num categories: 4
category id|name: 3415.0|Fresh whole chicken
------------------------------------------------------------------
num categories: 5
category id|name: 4850.0|Frozen fish and seafood
------------------------------------------------------------------
num categories: 6
category id|name: 9972.0|Ham, excluding canned
------------------------------------------------------------------
num categories: 7
category id|name: 5846.0|Shelf stable fish and seafood
------------------------------------------------------------------
num categories: 8
category id|name: 290.0|Admission to movies, theaters, and concerts
------------------------------------------------------------------
num categories: 9
category id|name: 9102.0|Bacon, breakfast sausage, and related products
------------------------------------------------------------------
num categorie

[I 2024-09-07 17:19:16,111] Trial 0 finished with value: 0.002786771145962835 and parameters: {'loss_coef_1': 8.939349663601896e-09, 'loss_coef_2': 1.0974609261085547e-12, 'Lr': 6.876678183321973e-05}. Best is trial 0 with value: 0.002786771145962835.


...

num categories: 1
category id|name: 9540.0|Bacon and related products
------------------------------------------------------------------
num categories: 2
category id|name: 8302.0|Breakfast sausage and related products
------------------------------------------------------------------


/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python

num categories: 3
category id|name: 2135.0|Fresh and frozen chicken parts
------------------------------------------------------------------
num categories: 4
category id|name: 3415.0|Fresh whole chicken
------------------------------------------------------------------
num categories: 5
category id|name: 4850.0|Frozen fish and seafood
------------------------------------------------------------------
num categories: 6
category id|name: 9972.0|Ham, excluding canned
------------------------------------------------------------------
num categories: 7
category id|name: 5846.0|Shelf stable fish and seafood
------------------------------------------------------------------
num categories: 8
category id|name: 290.0|Admission to movies, theaters, and concerts
------------------------------------------------------------------
num categories: 9
category id|name: 9102.0|Bacon, breakfast sausage, and related products
------------------------------------------------------------------
num categorie

[W 2024-09-08 22:20:41,353] Trial 31 failed with parameters: {'loss_coef_1': 9.20066455964218e-15, 'loss_coef_2': 9.431221118812895e-07, 'Lr': 0.05186919368257036} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_65815/79448088.py", line 60, in objective
    min_error = training_and_evaluation_optuna(trial, model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, alpha, saving_param_path)
  File "/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/utils.py", line 276, in training_and_evaluation_optuna
    epoch_train_loss, epoch_train_mse_loss = training_loop_o

KeyboardInterrupt: 

In [ ]:
# Get the best trial
best_trial = study.best_trial

# Print the best trial
print('Best trial:')
print(f'Value: {best_trial.value}')
print('Parameters:')
for key, value in best_trial.params.items():
    print(f'{key}: {value}')

Trial 34 finished with value: 0.001336801915051069 and parameters: {'loss_coef_1': 3.5415170416798804e-07, 'loss_coef_2': 3.838248652722512e-12}. Best is trial 34 with value: 0.001336801915051069.


In [ ]:
EPOCHS = 50
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.GPSampler())
study.optimize(objective, n_trials=50)

Trial 12 finished with value: 0.0013367897678554235 and parameters: {'loss_coef_1': 3.592202479418775e-14, 'loss_coef_2': 1.3417253313109117e-10}. Best is trial 12 with value: 0.0013367897678554235.


In [ ]:
# Get the best trial
best_trial = study.best_trial

# Print the best trial
print('Best trial:')
print(f'Value: {best_trial.value}')
print('Parameters:')
for key, value in best_trial.params.items():
    print(f'{key}: {value}')